# Import Packages

In [1]:
# Method #1 GARCH Model
from arch import arch_model
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from IPython.display import display, HTML
from sklearn.metrics import mean_squared_error
from set_params import func_garch_train_test_split, count_train_test
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random as rd
import warnings

def calculate_iqr(values):
    # Calculate Q1
    Q1 = np.percentile(values, 25)
    # Calculate Q3
    Q3 = np.percentile(values, 75)
    # Calculate IQR
    IQR = Q3 - Q1
    return IQR

def detect_outliers_iqr(values):
    # Calculate the IQR of the values
    IQR = calculate_iqr(values)
    # Calculate Q1 and Q3
    Q1 = np.percentile(values, 25)
    Q3 = np.percentile(values, 75)
    # Define the lower and upper bound for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # Return a boolean array: True if the value is an outlier, False otherwise
    return lower_bound, upper_bound


# display(HTML("<style>.container { width:80% !important; }</style>"))
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)

# Data Processing

In [3]:
base_FTSE_df = pd.read_csv('../data/1.3-FTSE_Monthly_ESG_Volatility_Final.csv')
base_FTSE_df = base_FTSE_df.rename(columns={'Date_x':'date_key'})

In [5]:
train_df, valid_df, test_df = func_garch_train_test_split(validation = False, 
                                                    threshold = 24)

In [24]:
count_rows_df = count_train_test(train_df, test_df)

In [25]:
coverage_df = pd.read_csv('../Data/coverage_dataframe.csv')
coverage_df.PermID = coverage_df.PermID.astype(int)
coverage_df = coverage_df[['PermID', 'Name']]
coverage_df = coverage_df.rename(columns={'PermID':'Asset'})

In [26]:
train_df = pd.merge(train_df, coverage_df, how = 'left', on = 'Asset')
train_df.index = train_df.month_key

In [27]:
train_df.head(2)

,date_key,month_key,Asset,Open,High,Low,Close,Return,buzz,ESG,ESGCombined,ESGControversies,EnvironmentalPillar,GovernancePillar,SocialPillar,CSRStrategy,Community,Emissions,EnvironmentalInnovation,HumanRights,Management,ProductResponsibility,ResourceUse,Shareholders,Workforce,V^CC,V^RS,V^YZ,Name
month_key,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-11-01,2019-12-31,2019-11-01,5021764927,2.396,2.40187,2.370,2.372000,-0.005034,6.0,36.0,32.0,28.0,74.0,41.0,34.0,67.0,12.0,67.0,86.0,60.0,3.0,58.0,70.0,41.0,51.0,0.019451,0.016788,0.020533,M&G PLC
2019-12-01,2020-01-31,2019-12-01,5021764927,2.454,2.45400,2.372,2.402001,-0.009076,534.5,37.0,26.0,16.0,66.0,33.0,10.0,71.4,1.0,85.0,93.0,4.0,21.0,35.0,19.0,69.0,6.0,0.009869,0.013527,0.014436,M&G PLC


# Modeling

In [28]:
def compile_train_test_garch(train_df, test_df, sample = True, cap = False, viz = False):
    '''
    '''

    mresults = pd.DataFrame()
    q, p = 1,1
    
    if sample:
        assets = [4295894970, 8589934212]
    else:
        assets = train_df.Asset.unique().tolist()

    for r, asset in enumerate(assets): 
        
        rolling_predictions = []
        test_size = test_df[test_df['Asset'] == asset].shape[0]
        
        y_volatility = train_df[train_df['Asset'] == asset]['V^YZ']
        y_volatility.append(test_df[test_df['Asset'] == asset]['V^YZ'])
        y_test = y_volatility[-test_size:]
        name = train_df[train_df['Asset'] == asset].iloc[0,-1]
        
        for i in range(test_size):
            # train data
            y_train = y_volatility[:-(test_size-i)]
            model = arch_model(y_train, p=p, q=q)
            model_fit = model.fit(disp='off')
            # test data
            pred = model_fit.forecast(horizon=1)
            pred = np.sqrt(pred.variance.values[-1,:][0])
            
            # capping the outliers
            if cap:
                lower_bound, upper_bound = detect_outliers_iqr(y_train)
                if upper_bound < pred:
                    pred = upper_bound
            
            rolling_predictions.append(pred)

        indices = test_df[test_df.Asset == asset].index
        rolling_predictions = pd.Series(rolling_predictions, index=indices)
        # print(indices)
        
        mse_million = mean_squared_error(y_test,rolling_predictions)*10**3
        mresult = pd.DataFrame({
            'Asset': asset,
            'Name': name,
            'Model':'GARCH(1,1)',
            'Test Size': test_size,
            'MSE^3':mse_million
                    }
            , index=[r]
        )
        mresults = pd.concat([mresults, mresult])
        
        if viz:
            plt.figure(figsize=(10,4))
            true, = plt.plot(test_df[test_df.Asset == asset]['V^YZ'])
            preds, = plt.plot(rolling_predictions)
            plt.title(f'GARCH(1,1) {name}', fontsize=15)
            plt.legend(['True Volatility', 'Predicted Volatility'], fontsize=9)
            plt.xticks(rotation=45)
            plt.savefig(f'../Outputs/Garch/{str(r+1).zfill(3)}_GARCH-{name}.png')
            plt.close()
        
    return mresults

In [36]:
mresults = compile_train_test_garch(train_df, test_df, sample=False, cap = False, viz = False)

/usr/local/lib/python3.9/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.9/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.9/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.9/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.9/dist-packages/ar

---

# Results

## 24MONTH NO CAP

In [13]:
mresults.sort_values('MSE^3', ascending=False).head(5)

,Asset,Name,Model,Test Size,MSE^3
62,4295894970,Bunzl plc,"GARCH(1,1)",51,80.352324
18,4295897861,Micro Focus International Ltd,"GARCH(1,1)",10,2.096486
79,4295895061,Sibanye Uk Ltd,"GARCH(1,1)",18,0.990881
41,5036227579,EVRAZ plc,"GARCH(1,1)",19,0.950165
128,4295894987,British Airways PLC,"GARCH(1,1)",17,0.823081


In [14]:
np.mean(mresults['MSE^3'])

0.7277253901281328

## 24MONTH WITH CAP

In [20]:
mresults.sort_values('MSE^3', ascending=False).head(5)

,Asset,Name,Model,Test Size,MSE^3
18,4295897861,Micro Focus International Ltd,"GARCH(1,1)",10,1.782181
79,4295895061,Sibanye Uk Ltd,"GARCH(1,1)",18,1.020862
41,5036227579,EVRAZ plc,"GARCH(1,1)",19,0.950165
128,4295894987,British Airways PLC,"GARCH(1,1)",17,0.823081
31,4295898044,Ashtead Group PLC,"GARCH(1,1)",32,0.758230


In [21]:
np.mean(mresults['MSE^3'])

0.22413324492142586

## 36MONTH NO CAP

In [37]:
mresults.sort_values('MSE^3', ascending=False).head(5)

,Asset,Name,Model,Test Size,MSE^3
51,4295894970,Bunzl plc,"GARCH(1,1)",51,80.352324
67,4295895061,Sibanye Uk Ltd,"GARCH(1,1)",18,0.990881
31,5036227579,EVRAZ plc,"GARCH(1,1)",19,0.950165
111,4295894987,British Airways PLC,"GARCH(1,1)",17,0.823081
22,4295898044,Ashtead Group PLC,"GARCH(1,1)",32,0.758230


In [38]:
np.mean(mresults['MSE^3'])

0.7667764680687831

## 36MONTH WITH CAP

In [33]:
mresults.sort_values('MSE^3', ascending=False).head(5)

,Asset,Name,Model,Test Size,MSE^3
67,4295895061,Sibanye Uk Ltd,"GARCH(1,1)",18,1.020862
31,5036227579,EVRAZ plc,"GARCH(1,1)",19,0.950165
111,4295894987,British Airways PLC,"GARCH(1,1)",17,0.823081
22,4295898044,Ashtead Group PLC,"GARCH(1,1)",32,0.758230
78,5000047647,Friends Life FPG Ltd,"GARCH(1,1)",12,0.713776


In [34]:
np.mean(mresults['MSE^3'])

0.2017985825595635

---

In [39]:
threshold = str(36)
CAP = 'NCAP'
mresults.to_excel(f'../Results/1-GARCH-{threshold}MONTH-{CAP}.xlsx', index=None)